#Bytearrays and Bytestrings

Although text is better handled as Unicode we inevitably end up communicating stream of bytes. Internally Python represents all Unicode characters using the same sized object.

Depending on how your interpreter was built it might use a 16-bit or a 32-bit representation (most interpreters use UCS-2 to conserve memory, but thhis involves certain complexities). The fixed-size format makes it very easy to compute positions within a string, but would be impossibly wasteful as the medium of communication.

Ultimately, then, to transmit a Unicode string it will typically be somehow _encoded_. Both ends of a communication have to agree on which encoding will be used. The sender encodes the text to produce a string of bytes which are sent across to the recipient who must then _decode_ the stream to reproduce the Unicode string that was sent.

The _`bytes`_ (sometimes called _byte string_) and _`bytearray`_ types were introduced to allow explicit handling of binary data, decoding in to Unicode and encoding it on output. When you read data from a network stream or a file opened in binary mode the calls return sequences of bytes.

The Python2 backport isn't an unqualified success.

When we talk about “opening a file” in Python we generally think about reading text, though files can also be opened in binary mode. In older version of Python both modes returned sequences of bytes, and Python's primary string type was ASCII-oriented. Nowadays reading a binary file and reading a text file return two different types of object. See what happens when you read the source of this notebook in text mode. (We use this file because it must be available for you to be reading this notebook).

In [ ]:
f = open("bytearrays-and-bytestrings.ipynb", "r")
bs = f.read(10)
type(bs)

So reading a text file returns a string, but no Unicode conversion takes place. This has to be performed explicitly (in Python 3, all strings are Unicode).

When you open the same file in binary mode, therefore, you will find that the file's read methods also return a string object in Python 2.

In [ ]:
f = open("bytearrays-and-bytestrings.ipynb", "rb")
bs = f.read(90)
type(bs)

__Bytes Literals__  
When you want to include a bytes value in your program source you can use a _bytes literal_. These are very similar to the standard string literals such as `"this is a string"` and `'''this is another'''`, but the opening string quote is preceded by the letter `b`. This is why they are sometimes called byte strings. The interpreter represents these values the same way.

In [ ]:
bs

Even the escape sequences work the same way. You can see above the newline characters represented as `"\n"`. Don't expect Unicode escapes to work, though. The interpreter doesn't recognize them when it's scanning a string literal, and treats the escape sequence as individual characters. The interpreter correctly represents the backslash in its escaped form.

In [ ]:
b'\u20ac', len(b'\u20ac')

Since Python naturally represents bytes objects as a string of characters, in Python 2 indexing a byte string returns character (in Python 3 it reurns an integer!)

In [ ]:
bs[0]

This is, of course, the integer representation of the first character of the file, which fortunately happens to be an ASCII character. We have to apply the `ord()` function to convert it to an integer.

In [ ]:
ord(bs[0])

Although the ASCII set stops at `chr(127)` a byte can hold 256 different values. Such byte values can be entered as escape sequences just as they can with regular strings. In this next cell we use numeric encoding of 10 (octal 12, hex 0a), and the interpreter confirms that the character is indeed a newline.

In [ ]:
b'\012\x0a'

This is one way to enter character values higher than 127 into bytes literals. Of course, by programming you can generate values quite easily. And, while it might not be the preferred style, you can use `B"..."` rather than `b"..."`. (In Python 3 the `ord()` builtin returns the same type as indexing for bytestring arguments, which text strings don't do).

In [ ]:
b"\377\0xfe"+"".join([chr(i) for i in range(128, 137)])

In [ ]:
ord(b" "), B" "[0], ord(" "), " "[0]

In [ ]:
bytes("hello")

The `bytes()` function  is really a type, but that is irrelevant here. It can be used with several different types of argument in Python 3, with different results.
In Python 2 its argument is coerced to a string first.

  * An integer argument `n` in Python 3 produces a string of $n$ zero bytes. In Python 2 the interger is converted to a string.
  * An iterable argument in Python 3 must yield only integer values in `range(256)`, and produces a string whose successive byte values are the successive integers. In Python 2 the iterable must produce characters.
  * A string argument will produce a sequence of bytes, each of which
    has the same `ord()` value as the corresponding character in the
    argument.
  * A Unicode value will cause conversion errors if any of the characters are non-ASCII.

In [ ]:
type(bytes)

In [ ]:
print bytes(23)
print "".join(chr(i) for i in range(ord('A'), ord('A')+26))
print bytes("£")
print bytes(u"£")

In [ ]:
print bytes('Please pay €9.99 at exit')

So why does the final (string) argument above cause an exception? Because it is a Unicode string, so there can be no guarantee that each character can be represented as a byte. When called with a string argument you have no way to tell `bytes()` _how to encode the string_. (In Python 3 you do so by naming an encoding as a second argument).

So you have to rely on the source encoding to correctly encode Unicode chaarcters as byte strings.

#### Warning: 

In [ ]:
print(bytes('Please pay €9.99 at exit'))

In [ ]:
print(bytes('Please pay €9.99 at exit').encode("UTF-16"))

In [ ]:
print(bytes(u'Please pay €9.99 at exit').encode("UTF-16"))

In [ ]:
print u'Please pay €9.99 at exit'.encode("UTF-16")

In [ ]:
print u'Please pay €9.99 at exit'.encode("UTF-8")

__Puzzler__  
So, here's a question: does the source for this notebook include any non-ASCII characters (bytes whose values are ≥ 128)? It isn't too difficult to answer that question for a short file (and Notebooks shouldn't be anywhere NEAR gigabytes long: you have been warned!) if you know about the built-in `any()` function, which returns True if any member of the sequence argument is True (without evaluating the rest).

The following expression shows there are no non-ASCII characters in the file.

In [ ]:
any((c>127) for c in open(
                    "bytearrays-and-bytestrings.ipynb", "rb")
                    .read())

Trying the same operation using text mode raises an exception in Python 3 because it's illegal to compare characters and numbers. In Python 2 the values in a bytestring are characters, so you get the same result whether you open the file in binary or text mode.

In [ ]:
any((c>127) for c in open(
                    "bytearrays-and-bytestrings.ipynb", "r")
                    .read())

In [ ]:
fl = open("bytearrays-and-bytestrings.ipynb", "rb")

In [ ]:
type("")

In [ ]:
[any((c>127) for line in fl for c in line)]

###Possible Discussions

* Do we care about Python 3?
* Do we distinguish between _text_ data and _bytes_ data?

###And, of course, whatever _you_ want ...